In [4]:
%autoreload 2
%reload_ext autoreload

In [58]:
import PySimpleGUI as sg
import logging
import ArgConfigParse
import sys
import textwrap
import constants
from pathlib import Path
# import logging
# import ArgConfigParse
# from pathlib import Path
import os
import fnmatch
import multiprocessing
import time
import csv
# import sys
import shutil

In [59]:
logger = logging.getLogger(__name__)
logger.root.setLevel('DEBUG')

In [60]:
def find_files(directory, pattern):
    '''generator: search directory and sub directories for a file
    
    Args:
        directory(`str` or pathlib.Path): directory to search
        pattern(`str`): pattern to search for (e.g. *.txt; *.csv)
    
    Yields:
        matching files
        '''
    for root, dirs, files in os.walk(directory):
        for basename in files:
            if fnmatch.fnmatch(basename, pattern):
                filename = Path(root)/Path(basename)
                yield filename

In [61]:
def doExit(message='unknown error in unknown module: BSoD!', exit_level=0, testing=False):
    logger = logging.getLogger(__name__)
    
    logger.info('exiting before program completion with exit code {}'.format(exit_level))
    print('progam exited due to errors -- see the logs')
    if not testing:
        sys.exit(0)

In [62]:
def mapHeaders(file_csv, expected_headers=[]):
    '''map an expected list of header values to their position in a csv
    accepts:
        file_csv (filename) - list containing CSV
        expected_headers (list) - list of headers to search for, ignoring all others
    '''
    logger = logging.getLogger(__name__)
    logger.debug('mapping headers')
    
    
    missing_headers = []
    header_map = {}
        
    try:
        csvHeader = file_csv[0]
    except IndexError as e:
        logger.warning('csv empty: {}'.format(e))
        return(False)
        
    logger.debug('checking for missing headers')
    for each in expected_headers:
        if each not in csvHeader:
            logger.debug('missing: {}'.format(each))
            missing_headers.append(each)

    if len(missing_headers) > 0:
        logging.warning(f'missing expected headers: {missing_headers}')
    for index, value in enumerate(csvHeader):
        if value in expected_headers:
            header_map[value] = index
            
    logger.debug('completed mapping headers')
    return(header_map, missing_headers)

In [63]:
def readCSV(csvFile):
    '''read a CSV file into an array'''
    csvFile = Path(csvFile).expanduser()
    file_csv = []
    try:
        with open(csvFile, 'r') as file:
            reader = csv.reader(file)
            for row in reader:
                file_csv.append(row)
    except (OSError, IOError) as e:
        logging.error(f'could not read file: {csvFile}')
        logging.error(f'error: {e}')
        return(False)
    
    return(file_csv)

In [64]:
def checkSentry(sentryFile, basePath):
    '''Check a path for a sentry file's existance
    
    Args:
        sentryFile(`str`): file to search for
        basePath(`str` or pathlib.Path): path to search for sentry file
    
    Returns:
        file if found or False if not found
    '''
    fileFound = False
    sentryFile = Path(sentryFile)
    basePath = Path(basePath)
        
    for filename in find_files(basePath, '*.txt'):
        if sentryFile.name in filename.name:
            logging.info(f'sentry file ({sentryFile}) found')
            fileFound = filename.parent
            break
    if not fileFound:
        logging.warning(f'sentry file ({sentryFile}) not found in {basePath}')
    return(fileFound)

In [10]:
def createDirectories(studentCSV, headers, studentRoot, subDirs):\
    '''create sub-directories for students based on student name, number
    
        /path/to/studentRoot/ClassOf-2028/Washington, George - 123456/subDir1/
        [-----studentRoot---][-------------Student Path-------------][-subdir-]
    Args:
        studentCSV(`list`): CSV containing headers `ClassOf`, `Student_Number` 
        studentRoot(`str` or pathlib.Path): root for student information
        subDirs(`list`): list of subdirs to create'''
    dirCheck = {}
    logging.debug('creating student directories as needed')
    for student in studentCSV[1:]:
        LastFirst = student[headers['LastFirst']]
        ClassOf = 'ClassOf-'+student[headers['ClassOf']]
        Student_Number = student[headers['Student_Number']]
        studentDir = f'{LastFirst} - {Student_Number}'    
        studentDir = studentRoot/ClassOf/studentDir
        dirCheck[studentDir] = {}
        for subDir in subDirs:
            gradeLevelDir = studentDir/subDir
            try:
                gradeLevelDir.mkdir(parents=True)
                action = 'created'
                logging.debug(f'created: {gradeLevelDir}')
            except (FileExistsError):
                action = None
            except Exception as e:
                logging.error(e)
                doExit(e, 1)
                
            
            dirCheck[studentDir][subDir] = action
    return(dirCheck)

In [12]:
def dict_to_keys(settings):
    '''convert INI keys into PySimpleGUI keys
    Args:
        settings(`dictionary`): single level dictionary {'key_one': 'value1', 'value_two': 'value 2'}
        
    Retruns:
        `dict: single level dictionary {'-KEY ONE-': 'value1', '-VALUE TWO-': 'value 2'}'''
    keys_to_elements = {}
    for key in settings:
        keys_to_elements[key] = '-'+key.replace('_', ' ').upper()+'-'
    
    return keys_to_elements

In [13]:
def keys_to_dict(settings):
    '''normalize PySimpleGUI keys back to INI friendly keys
    Args:
        settings(`dictionary`): single level dictionary {'-KEY ONE-': 'value1', '-VALUE TWO-': 'value 2'}
    
    Returns:
        `dict`: normalized INI frindly keys {'key_one': 'value1', 'value_two': 'value 2'}
    '''
    myDict = {}
    for key in settings:
        myDict[str(key).replace('-', '').replace(' ','_').lower()] = settings[key]
        
    return myDict

In [14]:
def write_back_config(updates, config, section):
    '''normalize PySimpleGUI variable keys and write back to argparser dictionary
    
       values that do not exist in the original configuration are ignored
    
    Args:
        updates(`dict`): dictionary of keys and values returned by PySG window
        config(`dict`): argparser nested INI style dictionary
        section(`str`): key for section in config dictionary
        
    Returns:
        tuple(dict, dict): (nested argparser style dictionary, ignored values)
        
    '''
    normalized_dict = keys_to_dict(updates)
    ignored = {}
    for key in normalized_dict:
        if key in config[section]:
            logging.debug(f'updating {section}[{key}]')
            config[section][key] = normalized_dict[key]
        else:
            logging.debug(f'ignoring key: {key}')
            ignored[key] = normalized_dict[key]
    
    return config, ignored

In [15]:
def create_settings_window(settings):
    '''basic settings window'''
    settings_keys_to_elements = dict_to_keys(settings)
   
    sg.theme('Material 2')
        
#     def TextLabel(text): return sg.Text(text+':', justification='r', size=((20, 1)) )
        
    layout = [[sg.T('Settings', font='Any 18')],
              [TextLabel('Google Drive Path'), sg.Input(key='-GOOGLE DRIVE-'), sg.FolderBrowse(initial_folder='/Volumes/', target='-GOOGLE DRIVE-')],
              [sg.B('Save'), sg.B('Exit')]]

    window = sg.Window('Settings', layout, keep_on_top=True, finalize=True)
    
    for key in settings_keys_to_elements:
        try:
            window[settings_keys_to_elements[key]].update(value=settings[key])
        except Exception as e:
            logging.warn(f'problem updating GUI window from settings: key = {key}')
            
    
    return window

In [25]:
 def TextLabel(text): return sg.Text(text+':', justification='r', size=((20, 1)) )

In [26]:
def create_settings_window(settings):
    '''basic settings window'''
    settings_keys_to_elements = dict_to_keys(settings)
   
    sg.theme('Material 2')
        
   
        
    layout = [[sg.T('Settings', font='Any 18')],
              [TextLabel('Google Drive Path'), sg.Input(key='-GOOGLE DRIVE-'), sg.FolderBrowse(initial_folder='/Volumes/', target='-GOOGLE DRIVE-')],
              [sg.B('Save'), sg.B('Exit')]]

    window = sg.Window('Settings', layout, keep_on_top=True, finalize=True)
    
    for key in settings_keys_to_elements:
        try:
            window[settings_keys_to_elements[key]].update(value=settings[key])
        except Exception as e:
            logging.warn(f'problem updating GUI window from settings: key = {key}')
            
    
    return window

In [31]:
def create_main_window(settings):
    '''main window'''
    sg.theme('Material 2')
    
    layout = [[sg.T('Main Window')],
              [sg.T('Stuff here')],
#               [sg.ScrolledTextBox(size=(80,40), font='Courier 11')],
              [TextLabel('Student Export'), sg.In(key='-STUDENT EXPORT-'), sg.FileBrowse(target='-STUDENT EXPORT-', 
                        initial_folder='~/Downloads/')],
              [sg.B('Run'), sg.B('Exit'), sg.B('Settings'), sg.B('Advanced Settings')]]
    
    return sg.Window('Main Application', layout)

In [32]:
def output(text, cols=65, indent=3):
    '''wrap and indent output to make it easier to read; print wrapped text
    Args:
        text(`str`): text to format
        cols(int): number of columns to wrap at
        indent(int): number of spaces to add to second-N lines
    Returns:
        `str`: wrapped and indended string with linebreaks'''
    wrapper = textwrap.TextWrapper()
    wrapper.width = cols
    wrapper.subsequent_indent = ' ' * indent
    text = wrapper.wrap(text)
    new_text = ''
    for i in text:
        new_text = new_text + i + '\n'
        
    print(new_text)
    return new_text

In [68]:
def create_folders(settings):
    basePath = settings['user_settings']['google_drive']
    sentryFile = settings['main']['sentry_file']
    csvFile = settings['input']['csv_file']
    expectedHeaders = [n.strip() for n in settings['main']['expected_headers'].rsplit(',')]
    try:
        myFile = open(settings['advanced_settings']['grade_level_dirs'], 'r')
        subDirs = myFile.read().rsplit('\n')
        subDirs = [n.strip() for n in subDirs]
    except OSError as e:
        doExit(e, 1)
    
    ## FIXME - Add a timeout around this
    studentRoot = checkSentry(sentryFile, basePath)
    if not studentRoot:
        logging.warning(f'This Google Shared Drive ({basePath}) does not appear to include the required sentry file: {sentryFile}')
        logging.warning('Try selecting a different Google Shared Drive.')
        doExit()
    
    ## FIXME handle false returns gracefully
    studentCSV = readCSV(csvFile)
    
    if not studentCSV:
        doExit(f'failed to read {csvFile}', 1)
    
    ## FIXME handle missing headers gracefully
    headers, missing = mapHeaders(studentCSV, expectedHeaders)

    log = createDirectories(studentCSV=studentCSV, headers=headers, studentRoot=studentRoot, subDirs=subDirs)
    
    
    results = {}
    actions = 0
    for student in log:
        created = 0
        for subdir in log[student]:
            if log[student][subdir]:
                created += 1
        results[student] = created
    for each in results:
        if results[each] > 0:
            print(f'created or updated:\n {each}\n  created: {results[each]} folders')
        else:
            print(f'no action needed for:\n {each}')    
    return(True)

In [71]:
parser = ArgConfigParse.ConfigFile(['./portfolioCreator.ini'])
parser.parse_config()
config = parser.config_dict
config['input'] = {}
config['input']['csv_file'] = Path('./student.export.text')
create_folders(config)

INFO:root:processing config files: [PosixPath('/Users/aaronciuffo/Documents/src/portfolioCreator/portfolioCreator.ini')]
INFO:root:sentry file (sentryFile_DO_NOT_REMOVE.txt) found
DEBUG:__main__:mapping headers
DEBUG:__main__:checking for missing headers
DEBUG:__main__:completed mapping headers
DEBUG:root:creating student directories as needed
DEBUG:root:created: /Volumes/GoogleDrive/Shared drives/IT Blabla I/Student Cumulative Folders (AKA Student Portfolios)/ClassOf-2023/Wanja, Michelle - 505586/00-Preschool
DEBUG:root:created: /Volumes/GoogleDrive/Shared drives/IT Blabla I/Student Cumulative Folders (AKA Student Portfolios)/ClassOf-2023/Wanja, Michelle - 505586/00-Transition Kindergarten
DEBUG:root:created: /Volumes/GoogleDrive/Shared drives/IT Blabla I/Student Cumulative Folders (AKA Student Portfolios)/ClassOf-2023/Wanja, Michelle - 505586/00-zKindergarten
DEBUG:root:created: /Volumes/GoogleDrive/Shared drives/IT Blabla I/Student Cumulative Folders (AKA Student Portfolios)/ClassOf

DEBUG:root:created: /Volumes/GoogleDrive/Shared drives/IT Blabla I/Student Cumulative Folders (AKA Student Portfolios)/ClassOf-2022/Jaber, Fahad - 505564/00-zKindergarten
DEBUG:root:created: /Volumes/GoogleDrive/Shared drives/IT Blabla I/Student Cumulative Folders (AKA Student Portfolios)/ClassOf-2022/Jaber, Fahad - 505564/01-Grade
DEBUG:root:created: /Volumes/GoogleDrive/Shared drives/IT Blabla I/Student Cumulative Folders (AKA Student Portfolios)/ClassOf-2022/Jaber, Fahad - 505564/02-Grade
DEBUG:root:created: /Volumes/GoogleDrive/Shared drives/IT Blabla I/Student Cumulative Folders (AKA Student Portfolios)/ClassOf-2022/Jaber, Fahad - 505564/03-Grade
DEBUG:root:created: /Volumes/GoogleDrive/Shared drives/IT Blabla I/Student Cumulative Folders (AKA Student Portfolios)/ClassOf-2022/Jaber, Fahad - 505564/04-Grade
DEBUG:root:created: /Volumes/GoogleDrive/Shared drives/IT Blabla I/Student Cumulative Folders (AKA Student Portfolios)/ClassOf-2022/Jaber, Fahad - 505564/05-Grade
DEBUG:root:cre

created or updated:
 /Volumes/GoogleDrive/Shared drives/IT Blabla I/Student Cumulative Folders (AKA Student Portfolios)/ClassOf-2023/Wanja, Michelle - 505586
  created: 15 folders
created or updated:
 /Volumes/GoogleDrive/Shared drives/IT Blabla I/Student Cumulative Folders (AKA Student Portfolios)/ClassOf-2022/Fordney, Joseph - 505567
  created: 15 folders
created or updated:
 /Volumes/GoogleDrive/Shared drives/IT Blabla I/Student Cumulative Folders (AKA Student Portfolios)/ClassOf-2022/Harvey, Emma - 505552
  created: 15 folders
created or updated:
 /Volumes/GoogleDrive/Shared drives/IT Blabla I/Student Cumulative Folders (AKA Student Portfolios)/ClassOf-2022/Jaber, Fahad - 505564
  created: 15 folders
created or updated:
 /Volumes/GoogleDrive/Shared drives/IT Blabla I/Student Cumulative Folders (AKA Student Portfolios)/ClassOf-2021/Lillie, Cameron - 505590
  created: 15 folders


True

In [33]:
def main():
    
    logfile = constants.appName+'.log'
    logfile = Path('~/').expanduser()/Path(logfile)

#     # file handler for all levels
#     fh = logging.handlers.RotatingFileHandler(logfile, mode='a', maxBytes=2000, backupCount=2)
#     fh.setLevel(logging.DEBUG)
#     fh.setFormatter(formatter)

#     logger.addHandler(fh)
    
    
    parser = ArgConfigParse.ConfigFile(['./portfolioCreator.ini'])
    parser.parse_config()
    config = parser.config_dict
    
    
    logger.setLevel(config['advanced_settings']['log_level'])
    
    window = create_main_window(config['user_settings'])
    settings_updated = False
    
    sg.easy_print('Starting up', font='Courier 11', do_not_reroute_stdout=False, no_button=True, size=(80, 40), location=(10,10))
    
    logger.debug('')
    
    while True:
        if window is None:
            window = create_main_window(config['user_settings'])            
        
        
#         window.Refresh()

        event, values = window.read()
                
        if event in (sg.WIN_CLOSED, 'Exit'):
            break
        
        if event == 'Run':
            settings['input']['csv_file'] = values['-STUDENT EXPORT-']
            pass
        
        if event == 'Settings':
            event, values = create_settings_window(config['user_settings']).read(close=True)
            
            if event == 'Save':
                window.close()
                window = None
                
                config, ignored = write_back_config(values, config, 'user_settings')
                
                output('updating settings')
                ArgConfigParse.write(config, './portfolioCreator.ini')                
        
        if event == 'Advanced Settings':
            event, values = create_adv_settings_window(config['advanced_settings']).read(close=True)
            
            logger.debug(values)
            
            if event == 'Save':
                window.close()
                window = None
                
                # handle no changes to this setting
                if len(values['-LOG LEVEL-'])<1:
                    values['-LOG LEVEL-'] = [config['advanced_settings']['log_level']]
                
                # write back any changes
                config, ignored = write_back_config(values, config, 'advanced_settings')
                # store only the first item in the list
                if isinstance(config['advanced_settings']['log_level'], list):
                    config['advanced_settings']['log_level'] = config['advanced_settings']['log_level'][0]
                
                
                output('saving advanced settings')
                ArgConfigParse.write(config, './portfolioCreator.ini')
                
    


    sg.easy_print_close()
    window.close()

In [34]:
main()

INFO:root:processing config files: [PosixPath('/Users/aaronciuffo/Documents/src/portfolioCreator/portfolioCreator.ini')]
DEBUG:__main__:
